# Ring Toss

Fivethirtyeight's Riddler asks "[how many ways can you stack](https://fivethirtyeight.com/features/can-you-make-24/) a [Rock-a-Stack](https://www.fisher-price.com/en-us/product/rock-a-stack-fgw58)?"

The only rule for stacking is that the $k^{th}$ smallest ring will stop at position $k$ (the top being position 1), so a ring out of place will necessarily decrease the number of rings we can stack. We'll number the rings $1, 2, \ldots, N$ from smallest to largest (for the main example, $N = 5$). 

Let $S_N$ be the number of ways to stack with $N$ rings. If we choose ring $k$ first, any larger ring fits on top. For example, if we start with ring 4, ring 5 or ring 3 will fit with no gaps. (We will, apparently, ignore the possibility that a particularly large ring fits entirely over a smaller ring.) Let $S_{N,k}$ be the number of possible stacks if we choose ring $k$ first (so $S_N = \sum_{k=1}^{N}S_{N,k}$). We can calculate this directly for some small values of $k$:

- If we choose ring 1, we're done. $S_{N,1} = 1$
- If we choose ring 2, we can choose any other ring to fit on top of it. $S_{N,2}$ = $N-1$
- If we choose ring 3, we can pick ring 1 next, and be done, or we can pick any of the other $N-2$ rings, then pick any of the remaining $N-2$ rings (including ring 1). $S_{N,3} = (N-2)^2 + 1$

From there, it starts getting complicated. But what we can see is that if we choose ring $k$, the smaller rings are equivalent to a $(k-1)$-ring stack. Further, choosing a larger ring next is effectively the same as choosing ring $k-1$ at this point, since it will fit in the same position, so the number of stacks we get if we choose the larger ring is the same as the number of stacks from choosing ring $k-1$. Effectively, this is like having $N-(k-1)$ copies of ring $k-1$. Put mathematically, 
$$S_{N,k} = (N-k+1)S_{N-1,k-1} + \sum_{i=1}^{k-2}S_{N-1,i}$$

In [1]:
function stackrings(N)
    numstacks = 0
    for k in 1:N
        numstacks += substacks(N,k)
    end
    return numstacks
end

function substacks(N, k)
    if k == 1
        return 1
    end
    return (N-k+1)*substacks(N-1,k-1) + sum([substacks(N-1,i) for i in 1:(k-2)])
end

substacks (generic function with 1 method)

In [2]:
[stackrings(N) for N in 1:10]

10-element Array{Int64,1}:
     1
     2
     5
    14
    43
   144
   523
  2048
  8597
 38486

Plugging these results into OEIS, we get [sequence A047970](https://oeis.org/A047970). No closed form for this sequence is listed, but it is the "antidiagonal sum of the [nexus numbers](https://oeis.org/A047969)", which *do* have a formula:
$$a(n,k) = (n+1)^{k+1} - n^{k+1}, n \geq 0, k \geq 0$$

If this is correct, then $S_N$ should satisfy:
$$S_N = \sum_{k=0}^{N-1} a(n-k-1, k)$$
(note that since we start at $N=1$, and the nexus numbers start at $n=0$, we are subtracting one from N in the formula)

For example, $S_4 = a(3,0) + a(2,1) + a(1,2) + a(0,3) = 1 + 5 + 7 + 1 = 14$.

Substituting the formula for $a(n,k)$, we get
$$S_N = \sum_{k=0}^{N-1} (N-k)^{k+1}-(N-k-1)^{k+1}$$
or
$$S_N = \sum_{k=1}^N (N-k+1)^{k}-(N-k)^{k}$$

Let's see if we get a match:

In [3]:
function nexus(n,k)
    return (n+1)^(k+1) - n^(k+1)
end

function stackrings2(N)
    return sum([nexus(N-k-1, k) for k in 0:(N-1)])
end

stackrings2 (generic function with 1 method)

In [4]:
[stackrings2(N) for N in 1:10]

10-element Array{Int64,1}:
     1
     2
     5
    14
    43
   144
   523
  2048
  8597
 38486

Huzzah! Kind of surprising OEIS doesn't list that formula.